# Predicting sale price of flat at Yishun Natura

We have a new flat coming up at a location closer to where my parents are staying. This makes it easier for us to take care of them, and also for them to meet our kids in the future. Howeer, as with all Singapore flats, given that they are subsidised (haevily) by the Singapore government, we will have to sell our existing flat at Yishun once we take possession of our new flat. 

I thought making this personal experience into a small project for my data science portfolio since this is something that I have vested interest in and by extension, motivated to work on. This file will contain the code and descriptions for the HDB project that I am doing as part of my data science portfolio.

### Inspiration from GovTech DSAID blog

A huge part of this project is inspired from a [blog](https://medium.com/dsaid-govtech/wanted-data-scientists-how-we-designed-our-first-case-interview-9fd4eaa4607c) written by the [Data Science and Artificial Intelligence Division (DSAID)](https://medium.com/dsaid-govtech) which I hope to get into one day. After seeing the impact they have on the life on Singaporeans, after launching useful products like the TraceTogether Token and App, I thought what they are doing is very meaningful and creates impact in our everyday lives. I am sure the job there will be sufficiently motivating in and of itself.

I would want to start off with using the OneMap API to geocode the lat-long coordinates just for the fun of it. Intuitively, I do not think that should affect the selling price anymore than the 'Town' feature. Including it is purely for practice on calling information from APIs.

First, the essential libraries...

In [2]:
import pandas as pd
import numpy as np
import requests as req
from matplotlib import pyplot as plt
import json

Based on the information on [HDB API](https://data.gov.sg/dataset/resale-flat-prices), we can find the API query string as follows. We will perform our entire analysis using our entire dataset of 92,270 entries(correct as of the time of writing).


In [3]:

query_str = 'https://data.gov.sg/api/action/datastore_search?resource_id=42ff9cfe-abe5-4b54-beda-c88f9bb438ee&limit=500'
response = req.get(query_str)
response.status_code

200

A side note on the response obtained from the **requests** library. A *status_code* of 200 implies that the request was successful. Otherwise the response code would have been *404 NOT FOUND*. Other possible status codes can be found [here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

In [10]:
data = json.loads(response.content)
type(data)

dict

In [12]:
len(data['result']['records'])

500

The number of entries here should be the same as what we have requested for above in the API query.